In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
pd.set_option('max_colwidth', 250)
import re
from bs4 import BeautifulSoup
from datasets import load_dataset

**In this notebook we compute embeddings of datasets to save them and use them later to train a sentiment model in another notebook** <br />
Most of these datasets are in spanish

In [12]:
datasets_path='../../data/sentiment_datasets/'
embeddings_path='../../data/sentiment_datasets_embeddings/'

## IMDB dataset

Dataset obtained from https://www.kaggle.com/arunmohan003/sentiment-analysis-using-lstm-pytorch/data

In [11]:
df_movies=pd.read_csv(datasets_path+'IMDB_Dataset.csv')
df_movies.shape

(50000, 2)

In [4]:
#Remove HTML characters
df_movies['review']=df_movies['review'].apply(lambda x: x.replace('<br />',' '))
df_movies.head(5)

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me. The first thing that struck me about Oz was its brutality and unflinching scenes of violen...",positive
1,"A wonderful little production. The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. The actors are extremely well chosen- Michael Sheen ...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the w...",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time. This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie. OK, fi...",negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different...",positive


In [5]:
#Model used for every dataset
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

In [30]:
#In case you want to retrain
#IMDB_Dataset_embeddings=np.array(model.encode(list(df_movies['review']), 
#                                            show_progress_bar=True,batch_size=64))
#with open(embeddings_path+'IMDB_Dataset_embeddings.npy', 'wb') as f:
#     np.save(f,IMDB_Dataset_embeddings)

Batches:   0%|          | 0/782 [00:00<?, ?it/s]

In [6]:
df_movies['sentiment'].head(5)

0    positive
1    positive
2    positive
3    negative
4    positive
Name: sentiment, dtype: object

In [13]:
#I save them with integer arrays and with their labels
label_dict={'positive':1,'negative':-1}
with open(embeddings_path+'IMDB_Dataset_labels.npy', 'wb') as f:
     np.save(f,np.array(df_movies['sentiment'].apply(lambda x: label_dict[x])))

In [14]:
#Embeddings saved in:
with open(embeddings_path+'IMDB_Dataset_embeddings.npy', 'rb') as f:
     IMDB_Dataset_embeddings=np.load(f)

# Open sentiment analysis dataset

Dataset from https://github.com/charlesmalafosse/open-dataset-for-sentiment-analysis

### ES teams

In [18]:
df_ES_teams=pd.read_csv(datasets_path+'betsentiment-ES-tweets-sentiment-teams.csv',encoding='latin-1').drop(\
                        ['tweet_id','tweet_date_created','language'],axis=1)
df_ES_teams=df_ES_teams.dropna(axis=0) #Quito rows con Nones
df_ES_teams=df_ES_teams[df_ES_teams.sentiment!='MIXED']#Quitamos tweets con la categoria mixed
df_ES_teams.loc[:,'sentiment_score']=df_ES_teams['sentiment_score'].apply(eval)#Paso de string a dict
df_ES_teams['sentiment_score']=df_ES_teams['sentiment_score'].apply(lambda x: max(x.values())) #Me quedo con la puntuación del sentimiento más probable
df_ES_teams=df_ES_teams[df_ES_teams.sentiment_score>0.65] #Me quedo con las que tengan una confianza mayor de 0.65
df_ES_teams.head()

,tweet_text,sentiment,sentiment_score
0,"Alisson puede estar más tranquilo, no cargará con el peso de ser el arquero mas caro de la historia. Si es que eso le pesaba en algo. Gracias @ChelseaFC",POSITIVE,0.745924
1,@iPincheViky @ChelseaFC Es que el director ejecutivo es mujer. So... Jajajajjaa.,NEUTRAL,0.827012
2,Upto £100 #freebets &gt; https://t.co/cbjeMXI99f #lol #gunners #matchedbetting #c4racing #cheltenhamfestival https://t.co/dg8clEuqku,NEUTRAL,0.930982
3,"Bobby Duncan, primo de Steven Gerrard, deja la cantera del #ManCity para firmar por el #Liverpool https://t.co/6uMcVrYmgM",NEUTRAL,0.906872
4,@TorreiraForeva @lepvtron @Arsenal @IntChampionsCup Leon bailey or Federico cheisa,NEUTRAL,0.942406


In [19]:
df_ES_teams.sentiment.value_counts()

NEUTRAL     91263
POSITIVE     5276
NEGATIVE     3380
Name: sentiment, dtype: int64

In [20]:
np.random.seed(42)
#Como hay pocos de negativo y positivo, cojo todos
ES_teams_negativo=df_ES_teams[df_ES_teams.sentiment=='NEGATIVE']
ES_teams_positivo=df_ES_teams[df_ES_teams.sentiment=='POSITIVE']
#Para neutro elijo la mitad de lo que busco y con el siguiente dataset haré la otra mitad
n=8000
ES_teams_neutral=df_ES_teams[df_ES_teams.sentiment=='NEUTRAL']
ES_teams_neutral=ES_teams_neutral.iloc[np.random.randint(0, len(ES_teams_neutral), n)]
ES_teams=pd.concat([ES_teams_positivo,ES_teams_negativo,ES_teams_neutral]).drop(['sentiment_score'],axis=1)

In [21]:
ES_teams.sentiment.value_counts()

NEUTRAL     8000
POSITIVE    5276
NEGATIVE    3380
Name: sentiment, dtype: int64

### ES World Cup

In [24]:
df_ES_worldcup=pd.read_csv(datasets_path+'betsentiment-ES-tweets-sentiment-worldcup-001.csv',encoding='latin-1').drop(\
                    ['tweet_id','tweet_date_created','language'],axis=1)
df_ES_worldcup=df_ES_worldcup.dropna(axis=0) #Quito rows con Nones
df_ES_worldcup=df_ES_worldcup[df_ES_worldcup.sentiment!='MIXED']#Quitamos tweets con al categoria mixed
df_ES_worldcup.loc[:,'sentiment_score']=df_ES_worldcup['sentiment_score'].apply(eval)#Paso de string a dict
df_ES_worldcup['sentiment_score']=df_ES_worldcup['sentiment_score'].apply(lambda x: max(x.values())) #Me quedo con la puntuación del sentimiento elegido
df_ES_worldcup=df_ES_worldcup[df_ES_worldcup.sentiment_score>0.65] #Me quedo con tweets con una confianza del sentimiento alta
df_ES_worldcup.head()

,tweet_text,sentiment,sentiment_score
1,"#MarioPereyraDT\n""Tenemos que jugarle a Francia con el mismo equipo"" https://t.co/15ccsyMtOi \n\n#SomosArgentina #Cadena3Mundial https://t.co/aNUZMjDkaR",NEUTRAL,0.878757
3,"Si llega a ser la despedida, no será la mejor...\nPero casi que quiero obligar a todos a mirar 4 años atrás y ver todo lo que, atrás de este tipo, lograron jugadores, dirigentes y todos los que forman parte de @Argentina \nGracias por hacernos me...",POSITIVE,0.686508
6,"Hoy no existen partidos políticos, sólo existe @miseleccionmx.",NEUTRAL,0.835554
7,"Hoy no existen partidos políticos, sólo existe @miseleccionmx.",NEUTRAL,0.835554
8,Festejan #cancunenses triunfo de la selección mexicana en #Rusia2018 @miseleccionmx ???? https://t.co/5pLA84L9MA,NEUTRAL,0.897296


In [25]:
df_ES_worldcup.sentiment.value_counts()

NEUTRAL     102166
POSITIVE     14227
NEGATIVE      7613
Name: sentiment, dtype: int64

In [26]:
np.random.seed(0)

#Para negativo cojo todas
ES_world_negativo=df_ES_worldcup[df_ES_worldcup.sentiment=='NEGATIVE']
#Para positivo cojo las necesarias para llegar a 15k
n=15000-len(ES_teams_positivo)
ES_world_positivo=df_ES_worldcup[df_ES_worldcup.sentiment=='POSITIVE']
ES_world_positivo=ES_world_positivo.iloc[np.random.randint(0, len(ES_world_positivo), n)]
#Para neutro cojo 8k
n=8000
ES_world_neutral=df_ES_worldcup[df_ES_worldcup.sentiment=='NEUTRAL']
ES_world_neutral=ES_world_neutral.iloc[np.random.randint(0, len(ES_world_neutral), n)]
ES_world=pd.concat([ES_world_positivo,ES_world_negativo,ES_world_neutral]).drop(['sentiment_score'],axis=1)


In [27]:
ES_world.sentiment.value_counts()

POSITIVE    9724
NEUTRAL     8000
NEGATIVE    7613
Name: sentiment, dtype: int64

In [32]:
#Uno los dos datasets
spanish_tweets=pd.concat([ES_world,ES_teams])
spanish_tweets.sentiment.value_counts()

NEUTRAL     16000
POSITIVE    15000
NEGATIVE    10993
Name: sentiment, dtype: int64

### Preprocesado de tweets

Hay que hacer una pequeña limpieza de los tweets para los embeddings

In [33]:
spanish_tweets.tweet_text=spanish_tweets.tweet_text.apply(lambda x: BeautifulSoup(x).get_text()) #Traduzco caracteres de HTML
#Quito cuentas y hashtags para no sesgar al modelo y asociar un sentimiento a alguna cuenta o hashtag en concreto por el dataset
spanish_tweets.tweet_text=spanish_tweets.tweet_text.apply(lambda x: re.sub("(@\S+)|(#\S+)", " ", x))
spanish_tweets.tweet_text=spanish_tweets.tweet_text.apply(lambda x: re.sub("(\w+:\/\/\S+)", " ", x)) #Quito enlaces
spanish_tweets.tweet_text=spanish_tweets.tweet_text.apply(lambda x: re.sub("\n", " ", x)) #Quito saltos de linea
spanish_tweets.tweet_text=spanish_tweets.tweet_text.apply(lambda x: ' '.join(x.split())) #Quito espacios extra
spanish_tweets.head()

,tweet_text,sentiment
37779,"Nunca me había levantado tan feliz a las 5:00 am, vamos que vamoooooooooooos",POSITIVE
151440,Me encanta lo que el fut logra hacer.,POSITIVE
137541,Vamos Argentina que hoy será un gran día! Con mucha fe que hoy se gana! ????????????????????????,POSITIVE
44939,Feliz cumpleaños a dos D10SES Que me hacen disfrutar del deporte mas lindo e interesante del ?... Gracias por tanto.,POSITIVE
183969,Ahora se viene mi selección francesa de fútbol. VAMOS CARAJO,POSITIVE


In [62]:
#En caso de querer reentrenar
#spanish_sentiment_embeddings=np.array(model.encode(list(spanish_tweets['tweet_text']), 
#                                           show_progress_bar=True,batch_size=128))
#with open(embeddings_path+'spanish_tweets_sentiment_embeddings.npy', 'wb') as f:
#     np.save(f,spanish_sentiment_embeddings)

Batches:   0%|          | 0/329 [00:00<?, ?it/s]

In [63]:
#Lo guardo también con sus etiquetas
label_dict={'POSITIVE':1,'NEUTRAL':0,'NEGATIVE':-1}
with open(embeddings_path+'spanish_tweets_sentiment_labels.npy', 'wb') as f:
     np.save(f,np.array(spanish_tweets['sentiment'].apply(lambda x: label_dict[x])))

In [64]:
#Los embeddings están en:
with open(embeddings_path+'spanish_tweets_sentiment_embeddings.npy', 'rb') as f:
     spanish_sentiment_embeddings=np.load(f)

## Hugging face datasets

Estos datasets se extraen de la librería *datasets* de Hugging Face

### Dataset de reviews de películas en español

In [36]:
dataset_reviews_es = load_dataset("muchocine")

Using custom data configuration default
Reusing dataset muchocine (C:\Users\marcb\.cache\huggingface\datasets\muchocine\default\1.1.1\3ed5582584cd84ef722606a3d725ef18fd4647d63195fef05c47683e5a056ccd)


  0%|          | 0/1 [00:00<?, ?it/s]

In [37]:
#El texto usado sera 'review_summary' por que 'review_body' es muy largo
es_movies=pd.DataFrame(dataset_reviews_es['train']).drop('review_body',axis=1)
# Paso las puntuaciones de 5 estrellas a 3 categorias. 3=NEUTRAl
es_movies.star_rating=pd.cut(es_movies.star_rating,3,labels=['NEGATIVE','NEUTRAL','POSITIVE']) 
es_movies.star_rating.value_counts()

POSITIVE    1350
NEGATIVE    1273
NEUTRAL     1249
Name: star_rating, dtype: int64

In [38]:
es_movies.head()

,review_summary,star_rating
0,"May, ¿quieres ser mi amigo?",POSITIVE
1,Cómo ponerse en la piel de un kamikaze,POSITIVE
2,"Silicona, esteroides, pactos demoníacos y otras basuras habituales son la base que sustentan esta aberración. De vergüenza.",NEGATIVE
3,Una comedia entretenida y poca cosa más para ver una tarde de domingo,NEGATIVE
4,"Luc Besson sabe manejar la acción, y aquí lo demuestra de nuevo manteniendo todo el film un ritmo trepidante sin apenas descanso.",POSITIVE


In [39]:
#Reentreno de embeddings
#es_movies_embeddings=np.array(model.encode(list(es_movies['review_summary']), 
#                                            show_progress_bar=True,batch_size=128))
#with open(embeddings_path+'es_movies_embeddings.npy', 'wb') as f:
#     np.save(f,es_movies_embeddings)

In [40]:
#Guardo también labels
#Lo guardo también con sus etiquetas
label_dict={'POSITIVE':1,'NEUTRAL':0,'NEGATIVE':-1}
with open(embeddings_path+'es_movies_labels.npy', 'wb') as f:
     np.save(f,np.array(es_movies['star_rating'].apply(lambda x: label_dict[x])))

In [ ]:
#Embeddings en:
with open(embeddings_path+'es_movies_embeddings.npy', 'rb') as f:
     es_movies_embeddings=np.load(f)

### Reviews de amazon en español

In [41]:
dataset_amazon = load_dataset("amazon_reviews_multi", "es")

Reusing dataset amazon_reviews_multi (C:\Users\marcb\.cache\huggingface\datasets\amazon_reviews_multi\es\1.0.0\724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609)


  0%|          | 0/3 [00:00<?, ?it/s]

In [32]:
dataset_amazon

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

In [21]:
pd.Series(dataset_amazon['train']['stars'][:40_001]).value_counts()

1    40000
2        1
dtype: int64

Los datos están ordenados por sus estrellas y hay 40k de cada una. Voy a elegir aleatoriamente 2.5k reviews de cada estrella excepto para las de 3 estrellas que cogeré 5k para tener dataset balanceado de negativo,neutro y positivo

In [42]:
np.random.seed(0)
num_1,num_2,num_3,num_4,num_5=(np.random.randint(0, 40_000,2_500),np.random.randint(40_000, 80_000,2_500),
                               np.random.randint(80_000, 120_000,5_000),np.random.randint(120_000, 160_000,2_500),
                               np.random.randint(160_000, 200_000,2_500))
nums=list(np.concatenate((num_1,num_2,num_3,num_4,num_5)))

In [43]:
es_reviews=pd.DataFrame(dataset_amazon['train'][nums])[['review_body','stars']]
es_reviews.stars=pd.cut(es_reviews.stars,3,labels=['NEGATIVE','NEUTRAL','POSITIVE']) # Paso a 3 categorias. 3=NEUTRAl

In [44]:
es_reviews.head()

,review_body,stars
0,"No ajusta correctamente, siempre se queda abierta y colgando. El segundo día se despegó la goma negra de la tapa plateada. Ya la he arrancado.",NEGATIVE
1,"Lo valoré anteriormente con una estrella por que me llegó roto. Posteriormente me envía el fabricante un mail diciéndome que quite la valoración negativa para intentar dar una solución al problema. Pues bien, entro para eliminar esa estrella y, S...",NEGATIVE
2,alguien tuvo problemas con la batería?? el teléfono hace un año en diciembre pero la batería ya no carga . El problema es que la garantía de la bateria es solamente de 6 meses.,NEGATIVE
3,Para lo que cuesta debería ser más rápido en cargar y más velocidad de carga. Me decepcionó bastante y estoy arrepentida en haberlo comprado,NEGATIVE
4,No es la consola que describen. Es el modelo normal.,NEGATIVE


In [35]:
#Reentreno de embeddings
#es_reviews_embeddings=np.array(model.encode(list(es_reviews['review_body']), 
#                                            show_progress_bar=True,batch_size=128))
#with open(embeddings_path+'es_reviews_embeddings.npy', 'wb') as f:
#     np.save(f,es_reviews_embeddings)

Batches:   0%|          | 0/118 [00:00<?, ?it/s]

In [46]:
#Guardo también labels
label_dict={'POSITIVE':1,'NEUTRAL':0,'NEGATIVE':-1}
with open(embeddings_path+'es_reviews_labels.npy', 'wb') as f:
     np.save(f,np.array(es_reviews['stars'].apply(lambda x: label_dict[x])))

In [47]:
#Los embeddings:
with open(embeddings_path+'es_reviews_embeddings.npy', 'rb') as f:
     es_reviews_embeddings=np.load(f)

## UOC tweets

Dataset extraido de https://github.com/jcsobrino/TFM-Analisis_sentimientos_Twitter-UOC

In [52]:
UOC_tweets=pd.read_csv(datasets_path+'global_dataset.csv',header=0,names=['Tweet', 'Sentimiento']).reset_index(drop=True)
UOC_tweets.shape

(73877, 2)

In [53]:
UOC_tweets=UOC_tweets[UOC_tweets.Sentimiento!='NONE'] #Quito los que no tienen sentimiento
UOC_tweets.Tweet=UOC_tweets.Tweet.apply(lambda x: re.sub("(@\S+)|(#\S+)", " ", x)) #Quito hashtags y menciones
UOC_tweets.Tweet=UOC_tweets.Tweet.apply(lambda x: re.sub("(\w+:\/\/\S+)", " ", x)) #Quito enlaces
UOC_tweets.Tweet=UOC_tweets.Tweet.apply(lambda x: ' '.join(x.split())) #Quito espacios extra
UOC_tweets.head()

,Tweet,Sentimiento
1,"Gonzalo Altozano tras la presentación de su libro 101 españoles y Dios. Divertido, emocionante y brillante.",P
2,"Mañana en Gaceta: TVE, la que pagamos tú y yo, culpa a una becaria de su falsa información sobre el cierre de",N
4,Más mañana en Gaceta. Amaiur depende de Uxue Barkos para crear grupo propio. ERC no cumple el req. del 15% y el PNV no quiere competencia,N
5,"Muy buenas noches followercetes, mañana va a ser un día bastante mitico para mi, ya os contare...",P
6,Más de mañana en Gaceta. UPyD contará casi seguro con grupo gracias al Foro Asturias. Eso se dice en el Congreso,P


In [54]:
UOC_tweets['Sentimiento'].value_counts()

P      26846
N      20110
NEU     3326
Name: Sentimiento, dtype: int64

In [55]:
#Voy a elegir este dataset para balancear las 3 clases obtenidas con otros datasets. Voy a coger todos de neutral, 3k de negativo y 1k de positivo
np.random.seed(0)
n=3000
UOC_tweets_negativo=UOC_tweets[UOC_tweets.Sentimiento=='N']
UOC_tweets_negativo=UOC_tweets_negativo.iloc[np.random.randint(0, len(UOC_tweets_negativo), n)]

n=1000
UOC_tweets_positivo=UOC_tweets[UOC_tweets.Sentimiento=='P']
UOC_tweets_positivo=UOC_tweets_positivo.iloc[np.random.randint(0, len(UOC_tweets_positivo), n)]

UOC_tweets_neutral=UOC_tweets[UOC_tweets.Sentimiento=='NEU']

UOC_tweets_filtrado=pd.concat([UOC_tweets_positivo,UOC_tweets_negativo,UOC_tweets_neutral])

In [56]:
UOC_tweets_filtrado.head()

,Tweet,Sentimiento
37545,"No hay más campeón q Arenas y el Premio Larra sentado a mi lado. RT El ""Gran Pablo"". Gracias x todo campeón",P
27540,"Argumentario del PP:""El PP gobierna sin mayoría absoluta en Aragón y Extremadura y ha podido aprobar los presupuestos con apoyo de otros""",P
9323,Ya es Presidente del Gobierno de España ;-)))))))))))))),P
15347,"“ Quiero dar la bienvenida a twitter a un genial director de cine (y sin embargo amigo), el gran +1!!!!;)",P
52341,esta el de Los Cabos.Qué bien me lo pasé en este concierto.,P


In [57]:
UOC_tweets_filtrado['Sentimiento'].value_counts()

NEU    3326
N      3000
P      1000
Name: Sentimiento, dtype: int64

In [46]:
#Reentreno de embeddings
#UOC_tweets_embeddings=np.array(model.encode(list(UOC_tweets_filtrado['Tweet']), 
#                                            show_progress_bar=True,batch_size=128))
#with open(embeddings_path+'UOC_tweets_embeddings.npy', 'wb') as f:
#     np.save(f,UOC_tweets_embeddings)

Batches:   0%|          | 0/58 [00:00<?, ?it/s]

In [60]:
#Guardo labels
label_dict={'P':1,'NEU':0,'N':-1}
with open(embeddings_path+'UOC_tweets_labels.npy', 'wb') as f:
     np.save(f,np.array(UOC_tweets_filtrado['Sentimiento'].apply(lambda x: label_dict[x])))

In [58]:
#Embeddings:
with open(embeddings_path+'UOC_tweets_embeddings.npy', 'rb') as f:
     UOC_tweets_embeddings=np.load(f)